In [2]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

# Basic Text Generation 

In [3]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model='gpt-5-mini',
    input='Write a one sentence prediction for the future of intelligence.'
)

print(response.output_text)

Intelligence will evolve into a globally distributed, hybrid ecosystem of human, artificial, and collective minds that amplifies creativity and problem-solving while forcing new social, ethical, and governance frameworks to manage its risks and distribution.


# Analyse Images and Files

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input=[
        {
            "role": "user",
            "content": [
             {
                 "type": "input_text",
                 "text": "What is in this image?"
             },
             {
                 "type": "input_image",
                 "image_url": "https://images.unsplash.com/photo-1464983953574-0892a716854b",
              }
             ]
         }
    ]
)


print(response.output_text)

A starry night landscape with a rugged, snow-capped mountain range. The foreground has patches of snow on dark slopes, and faint clouds sit low under a clear, starlit sky.


# Structured Outputs

In [12]:
from pydantic import BaseModel

class Tasks(BaseModel):
    tasks: list[str]
    deadlines: list[str]

meeting_notes = """
Yesterday we discussed several action items:
• John needs to review the Q2 budget proposal before the end of this week.
• Let's set up a kickoff meeting for the new project—how about next Monday?
• Anna will update and submit the marketing plan by April 15th.
Please make sure to update the status before our next check-in.
"""
response = client.responses.parse(
    model="gpt-5-mini",
    input=[
        {
            "role": "system",
            "content": "You Extract tasks and their deadlines from raw inputs."
        },
        {
            "role": "user",
            "content": meeting_notes
         }
    ],
    text_format=Tasks,
)

print(response.output_parsed)

tasks=['John: review the Q2 budget proposal', 'Set up a kickoff meeting for the new project', 'Anna: update and submit the marketing plan', 'Update status before our next check-in'] deadlines=['end of this week', 'next Monday', 'April 15th', 'before our next check-in']


# Function Calling

In [16]:
from openai import OpenAI
import json

client = OpenAI()

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "load_pdf_text",
        "description": "Load and return the full text from the provided PDF file path.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {
                    "type": "string",
                    "description": "Path to the PDF file to extract text from.",
                },
            },
            "required": ["file_path"],
        },
    },
]

def load_pdf_text(file_path):
    """Loads text from a PDF file."""
    from pypdf import PdfReader
    reader = PdfReader(file_path)
    text = "\n\n".join([page.extract_text() for page in reader.pages])
    return text

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "Can you read and summarize the file at ../assets/paper3.pdf?"}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "load_pdf_text":
            # 3. Execute the function logic for load_pdf_text
            file_args = json.loads(item.arguments)
            pdf_text = load_pdf_text(file_args["file_path"])

            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "pdf_text": pdf_text
                })
            })

print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-5",
    instructions="Summarize the PDF content provided via the tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
[{'role': 'user', 'content': 'Can you read and summarize the file at ../assets/paper3.pdf?'}, ResponseReasoningItem(id='rs_006bc7ff483635a100695cf77bc79c8197b0fec4105bde9a50', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"file_path":"../assets/paper3.pdf"}', call_id='call_cTsPsE9ORthI48yuXAdpPWB4', name='load_pdf_text', type='function_call', id='fc_006bc7ff483635a100695cf77ceb58819799344c8af232a3a9', status='completed'), {'type': 'function_call_output', 'call_id': 'call_cTsPsE9ORthI48yuXAdpPWB4', 'output': '{"pdf_text": "Ef\\ufb01cient Estimation of Word Representations in\\nVector Space\\nTomas Mikolov\\nGoogle Inc., Mountain View, CA\\ntmikolov@google.com\\nKai Chen\\nGoogle Inc., Mountain View, CA\\nkaichen@google.com\\nGreg Corrado\\nGoogle Inc., Mountain View, CA\\ngcorrado@google.com\\nJeffrey Dean\\nGoogle Inc., Mountain View, CA\\njeff@google.com\\nAbstract\\nWe propose two novel model archit

<img src="../assets/functions_schema.png" style="width: 50%;" />

https://platform.openai.com/docs/guides/function-calling#defining-functions

# Managing Conversation States

In [29]:
from openai import OpenAI
from openai.types.shared import reasoning_effort

client = OpenAI()

response = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {"role": "user", "content": "What are the 3 core skills for snowboarding?"},
    ],
    
)

print(response.output_text)

The 3 core skills for snowboarding are:

1. **Balance and Stance** – Maintaining proper body positioning and weight distribution on the board to stay stable and in control.

2. **Turning** – Learning how to carve and control direction primarily through edge control and shifting weight to execute smooth turns.

3. **Speed Control and Stopping** – Managing your speed safely using techniques like skidding, carving, and performing controlled stops.

Mastering these foundational skills helps build confidence and sets the stage for more advanced tricks and maneuvers.


In [30]:
# Add the response to the conversation
history = [
        {"role": "user", "content": "What are the 3 core skills for snowboarding?"},
    ]
history += [{"role": el.role, "content": el.content} for el in response.output]
history

[{'role': 'user', 'content': 'What are the 3 core skills for snowboarding?'},
 {'role': 'assistant',
  'content': [ResponseOutputText(annotations=[], text='The 3 core skills for snowboarding are:\n\n1. **Balance and Stance** – Maintaining proper body positioning and weight distribution on the board to stay stable and in control.\n\n2. **Turning** – Learning how to carve and control direction primarily through edge control and shifting weight to execute smooth turns.\n\n3. **Speed Control and Stopping** – Managing your speed safely using techniques like skidding, carving, and performing controlled stops.\n\nMastering these foundational skills helps build confidence and sets the stage for more advanced tricks and maneuvers.', type='output_text', logprobs=[])]}]

In [31]:
second_input = {"role": "user", "content": "For each of these give me 3 exercises to practice in simple bullets."}

history.append(second_input)

second_response = client.responses.create(
    model="gpt-4.1-mini",
    input=history,
)

print(second_response.output_text)

Certainly! Here are 3 exercises for each core snowboarding skill:

**1. Balance and Stance**  
- Practice balancing on one foot on flat ground (or on a balance board if available).  
- Side-to-side weight shifts while strapped into your board on gentle terrain.  
- Static holds: Standing still in your proper snowboarding stance with knees slightly bent and eyes forward.

**2. Turning**  
- Toe-edge and heel-edge slips down a gentle slope to feel the edges biting into the snow.  
- Linking wide S-shaped turns on a beginner slope, focusing on smooth weight transitions.  
- Traverse across the slope and gradually switch edges to initiate turning.

**3. Speed Control and Stopping**  
- Controlled falling leaf exercise: slide down while shifting weight to control speed without fully turning downhill.  
- Practice hockey stops by carving onto both edges and digging in to come to a quick stop.  
- Gradually increase speed on gentle terrain and practice controlled slow-down skidding turns.

Th

# Conversations API

To facilitate conversation management and avoid having to pass inputs manually like this, OpenAI created a special API to work with the Responses API called: "Conversations API".

The [Conversations API](https://platform.openai.com/docs/api-reference/conversations/create) works with the Responses API to persist conversation state as a long-running object with its own durable identifier. After creating a conversation object, you can keep using it across sessions, devices, or jobs.

Conversations store items, which can be messages, tool calls, tool outputs, and other data.

In [32]:

from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input="What are the 3 most important skills to develop as a programmer.",
)
print(response.output_text)

second_response = client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=response.id,
    input=[{"role": "user", "content": "Write down 2 exercises for each of those skills."}],
)
print(second_response.output_text)

As a programmer, several skills are crucial for success in the field. Here are three of the most important:

1. **Problem-Solving Skills**:
   - The ability to analyze complex problems, break them down into manageable parts, and devise effective solutions is essential. This involves logical thinking and creativity to address challenges and optimize code.

2. **Proficiency in Algorithms and Data Structures**:
   - Understanding algorithms and data structures is fundamental. This knowledge helps in writing efficient code, improving performance, and making informed decisions when choosing the right approach for a given problem.

3. **Collaboration and Communication**:
   - Programming is often a team effort. Strong communication skills facilitate better collaboration with team members, understanding requirements, and conveying ideas effectively. This includes documentation skills and the ability to provide constructive feedback. 

Focusing on these areas will greatly enhance your effectiv

In [ ]:
from openai import OpenAI

client = OpenAI()

# iniitliaze a conversation session with an ID
def create_conversation(user_id: str) -> str:
    # 1. Create a new conversation container
    conversation = client.conversations.create(
        metadata={"user_id": user_id, "status": "active"}
    )
    return conversation.id

# something to send messages
def send_message(conv_id: str, user_text: str) -> str:
    # 2. Send a turn through Responses, bound to that conversation
    response = client.responses.create(
        model="gpt-5-mini",
        input=[{"role": "user", "content": user_text}],
        conversation=conv_id,
        # store=True  # optional; default is stored for conversation state
    )
    return response.output_text

# ability to update conversation metadata
def update_tag_conversation(conv_id: str, topic: str) -> None:
    # 3. Update metadata on the conversation (for filtering, dashboards, etc.)
    client.conversations.update(
        conversation_id=conv_id,
        metadata={"topic": topic}
    )

user_id = "user-123"
conv_id = create_conversation(user_id)

msg = "Hi, I'm Lucas and I want help deciding on my new laptop. \
    Give me just a sentence with a simple decision making framework."
answer = send_message(conv_id, msg)
print("Assistant:", answer)

# maybe after inspecting the first message:
update_tag_conversation(conv_id, topic="shopping-help")
msg = "What's my name and what am I buying?"
followup = send_message(conv_id, msg)
print("Assistant:", followup)

# Managing Context

![Context window illustration](https://cdn.openai.com/API/docs/images/context-window.png)


- Context is managed in tokens
- Tokens are just simple units of text
- [Context window limits for each OpenAI Model](https://platform.openai.com/docs/models)

In [ ]:
import tiktoken

def get_num_tokens(prompt, model="gpt-5"):
    """Calculates the number of tokens in a text prompt"""
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(prompt))

In [35]:
get_num_tokens("Hello folks!")

3

## Token Cost & Context Windows for GPT-5-... models

__GPT-5.2__
- https://platform.openai.com/docs/models/gpt-5.2

__GPT-5.1__
- https://platform.openai.com/docs/models/gpt-5.1

__GPT-5__
- https://platform.openai.com/docs/models/gpt-5 

__GPT-5-mini__
- https://platform.openai.com/docs/models/gpt-5-mini